Imports

In [1]:
from ultralytics import YOLO
import cv2
import pandas as pd
from RealtimeTracking import *
import gc

Load the model

In [2]:
plate_det_model_path = '/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/models/YOLOv8/yolov8n_90e_cust/runs/detect/train4/weights/best.pt'
testvideo_path = '/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/video_data/testvideo_highway_mp4.mp4'

vehicle_det_model=YOLO('yolov8n.pt')
vehicle_ids = [2,3,5,7]
plate_det_model = YOLO(plate_det_model_path)

Capture on a video

In [3]:
def PlateToVehicle(det_license_plate,det_vehicles):
    lp_x1, lp_y1, lp_x2, lp_y2, lp_score, lp_class_id = det_license_plate
    found=False
    for i in range(len(det_vehicles)):
        v_x1, v_y1, v_x2, v_y2, v_id = det_vehicles[i]
        #inside
        if lp_x1 > v_x1 and lp_y1 > v_y1 and lp_x2 < v_x2 and lp_y2 < v_y2:
            v_index=i
            found=True
            break
    if found:
        return det_vehicles[v_index]
    return -1,-1,-1,-1,-1

def lpPrep(license_plate):
    lp_gray = cv2.cvtColor(license_plate,cv2.COLOR_BGR2GRAY)
    retval,lp_threshold = cv2.threshold(lp_gray,64,255,cv2.THRESH_BINARY_INV)
    return lp_threshold

def lpRead(license_plate):
    lp_Text='asd012'
    lp_Conf_score = 1
    return lp_Text, lp_Conf_score

In [4]:
gc.collect()

18

In [5]:
df_cols = ['fr_number','v_id', 'v_bbox','lp_bbox','lp_bbox_score','lp','lp_score']
df_rows = []

# load using Sort
video_capture = cv2.VideoCapture(testvideo_path)


mot_trckr = Sort()
results = {}
ret = True
frame_indexer=0
while ret:
    if (video_capture.isOpened()== False):
        print("Error opening video stream or file")
    
    ret, frame = video_capture.read()
    
    if ret and frame_indexer<10:
        results[frame_indexer] = {}
        #det
        det_objs = vehicle_det_model(frame)[0]
        vehicles=[]
        for det_obj in det_objs.boxes.data.tolist():
            obj_x1, obj_y1, obj_x2, obj_y2, obj_score, obj_class_id = det_obj
            if int(obj_class_id) in vehicle_ids:
                vehicles.append([obj_x1, obj_y1, obj_x2, obj_y2, obj_score])
                #print(obj_class_id)
        
        track_ids = mot_trckr.update(np.asarray(vehicles))
        #print(track_ids)
        #plates=[]
        det_lps = plate_det_model(frame)[0]
        for det_lp in det_lps.boxes.data.tolist():
            lp_x1, lp_y1, lp_x2, lp_y2, lp_score, lp_class_id = det_lp
            
            #assign license plate to vehicle
            vehicle_x1,vehicle_y1,vehicle_x2,vehicle_y2,vehicle_id = PlateToVehicle(det_lp,track_ids)
            if vehicle_id is not -1:
                #crop
                license_plate = frame[int(lp_y1):int(lp_y2),int(lp_x1):int(lp_x2),:]
                prepared = lpPrep(license_plate)
                #cv2.imshow('lp',license_plate)
                #cv2.imshow('lpprep',prepared)
                #cv2.waitKey(0)

                lp_text, lp_text_confscore = lpRead(prepared)
                if lp_text is not None:
                    df_rows.append({'fr_number':frame_indexer,
                                    'v_id':int(vehicle_id+1),
                                    'v_bbox':[vehicle_x1,vehicle_y1,vehicle_x2,vehicle_y2],
                                    'lp_bbox':[lp_x1, lp_y1, lp_x2, lp_y2],
                                    'lp_bbox_score':lp_score,
                                    'lp':lp_text,
                                    'lp_score':lp_text_confscore})
                
            
    
    frame_indexer+=1

<>:38: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:38: SyntaxWarning: "is not" with a literal. Did you mean "!="?
[ WARN:0@2.659] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@2.659] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin0 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@2.659] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipelin

In [6]:
out = pd.DataFrame(df_rows,columns=df_cols)
out

,fr_number,v_id,v_bbox,lp_bbox,lp_bbox_score,lp,lp_score
0,0,5,"[252.07601928710938, 549.6202392578125, 575.88...","[355.4010314941406, 719.619140625, 461.6276550...",0.467115,asd012,1
1,0,2,"[858.7146606445312, 616.114501953125, 1221.752...","[986.8820190429688, 815.9974365234375, 1114.75...",0.333886,asd012,1
2,0,3,"[980.7061767578125, 900.4325561523438, 1461.83...","[1159.7911376953125, 1149.2432861328125, 1293....",0.308768,asd012,1
3,1,5,"[252.11460278559866, 549.6452530102526, 575.88...","[355.40081787109375, 719.6344604492188, 461.58...",0.467120,asd012,1
4,1,2,"[858.4844545261479, 616.0321937067034, 1221.94...","[986.96044921875, 816.00048828125, 1114.767578...",0.333503,asd012,1
5,1,3,"[980.7313426363543, 900.4754747630882, 1461.75...","[1159.7852783203125, 1149.1611328125, 1293.970...",0.307354,asd012,1
6,2,5,"[252.0693792846115, 549.6147175445177, 575.898...","[355.51629638671875, 719.6675415039062, 461.61...",0.467570,asd012,1
7,2,2,"[858.3735940702186, 616.0541602149467, 1222.04...","[987.1072998046875, 815.9970703125, 1114.69360...",0.332931,asd012,1
8,2,3,"[980.8782300976031, 900.4811413778303, 1461.93...","[1160.0576171875, 1149.122314453125, 1293.8375...",0.307911,asd012,1
9,3,5,"[251.91572972634643, 549.567868758543, 575.840...","[355.5184326171875, 719.661376953125, 461.6175...",0.467799,asd012,1


In [7]:
#out.to_csv('df_log.csv')